In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.callbacks import EarlyStopping 
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV


In [2]:
# Importando os arquivos JSON
df = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\applicants.json").T
df_prospects = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\prospects.json").T
df_vagas = pd.read_json(r"C:\Users\Usuário\OneDrive\12. Programing\03. FIAP\02. Disciplinas\05. Deep Learning & Unstrutured Data\00. Datathon\01. Dados\vagas.json").T

In [3]:
# Tratando todos os dados

cols_com_dicionario =['infos_basicas','informacoes_pessoais','informacoes_profissionais','formacao_e_idiomas','cargo_atual']

dfs_normalizados = []

for col in cols_com_dicionario:
    df_temp = pd.json_normalize(df[col])
    df_temp.columns = [f"{col}_{subcol}" for subcol in df_temp.columns]
    dfs_normalizados.append(df_temp)

df_normalizado = pd.concat([df.drop(columns=cols_com_dicionario)] + dfs_normalizados, axis=1)

# "Abrindo" os dados de applicants
df_prospects.reset_index(inplace=True)
df_prospects.rename(columns={'index': 'id_vaga'}, inplace=True)

cols_com_dicionario_prospects =['prospects']

dfs_normalizados_prospects = []

for col in cols_com_dicionario_prospects:
    df_temp_prospects = pd.json_normalize(df_prospects[col])
    df_temp_prospects.columns = [f"{col}_{subcol}" for subcol in df_temp_prospects.columns]
    dfs_normalizados_prospects.append(df_temp_prospects)

df_normalizado_prospects = pd.concat([df_prospects.drop(columns=cols_com_dicionario_prospects)] + dfs_normalizados_prospects, axis=1)

cols_com_dicionario_prospects_melted = ['dict_prospect']

df_normalizados_prospects_melted2 = []

df_normalizado_prospects_melted = df_normalizado_prospects.melt(
    id_vars=['titulo', 'modalidade','id_vaga'],
    var_name='n_prospect',
    value_name='dict_prospect'
)
df_normalizado_prospects_melted.sort_values('id_vaga')

cols_com_dicionario_prospects_melted = ['dict_prospect']

df_normalizados_prospects_melted2 = []

for col in cols_com_dicionario_prospects_melted:
    df_temp_prospects_melted = pd.json_normalize(df_normalizado_prospects_melted[col])
    df_temp_prospects_melted.columns = [f"{col}_{subcol}" for subcol in df_temp_prospects_melted.columns]
    df_normalizados_prospects_melted2.append(df_temp_prospects_melted)
    
dfs_normalizados_prospects_melted2 = pd.concat([df_normalizado_prospects_melted.drop(columns=cols_com_dicionario_prospects_melted)] + df_normalizados_prospects_melted2, axis=1)

colunas_comuns = set(df_normalizado.columns).intersection(dfs_normalizados_prospects_melted2.columns)

# Analisando as tabelas de "vagas"

pd.set_option('display.max_columns', None)
cols_com_dicionario_df_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']

df_vagas_normalizado = []

for col in cols_com_dicionario_df_vagas:
    df_temp_vagas = pd.json_normalize(df_vagas[col])
    df_temp_vagas.columns = [f"{col}_{subcol}" for subcol in df_temp_vagas.columns]
    df_vagas_normalizado.append(df_temp_vagas)
    
df_vagas_normalizado = pd.concat([df_vagas.drop(columns=cols_com_dicionario_df_vagas)] + df_vagas_normalizado, axis=1)

df_vagas.reset_index(inplace=True)
df_vagas.rename(columns={'index': 'id_vaga'}, inplace=True)

#pd.set_option('display.max_columns', None)
cols_com_dicionario_df_vagas = ['informacoes_basicas', 'perfil_vaga', 'beneficios']

df_vagas_normalizado = []

for col in cols_com_dicionario_df_vagas:
    df_temp_vagas = pd.json_normalize(df_vagas[col])
    df_temp_vagas.columns = [f"{col}_{subcol}" for subcol in df_temp_vagas.columns]
    df_vagas_normalizado.append(df_temp_vagas)
    
df_vagas_normalizado = pd.concat([df_vagas.drop(columns=cols_com_dicionario_df_vagas)] + df_vagas_normalizado, axis=1)
# Transformando os 3 dataframes em um só

df_final = pd.merge(
    dfs_normalizados_prospects_melted2,  # DataFrame da esquerda (o principal)
    df_vagas_normalizado,                # DataFrame da direita (de onde vêm as novas colunas)
    on='id_vaga',                        # Coluna usada como chave para a junção
    how='left'                           # Tipo de junção
)

df_normalizado = df_normalizado.dropna(subset=['infos_basicas_codigo_profissional'])

df_final_total = pd.merge(
    df_final,  # DataFrame da esquerda (o principal)
    df_normalizado, # DataFrame da direita (de onde vêm as novas colunas)
    left_on='dict_prospect_codigo',                        # Coluna usada como chave para a junção
    right_on='infos_basicas_codigo_profissional',
    how='left'                           # Tipo de junção
)

df_final_total

,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,CONSULTOR CONTROL M,,4530,prospects_0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ P

In [ ]:
# Criando as feature engeneering
# Para aprimorar o modelo, criamos alguns matches, como, inglÊs, espanhol, academico e das descrições

## Match linguagem (inglês e espanhol)

# Passo 1: Criar o Mapa de Níveis Ordenados (o mesmo para ambos os idiomas)
mapa_niveis = {
    'Nenhum': 0,
    '': 0,
    np.nan: 0,
    'Básico': 1,
    'Intermediário': 2,
    'Técnico': 2,
    'Avançado': 3,
    'Fluente': 4
}

# --- Processamento para o Inglês ---

# Passo 2a: Converter as colunas de Inglês para números
nivel_candidato_ingles_num = df_final_total['formacao_e_idiomas_nivel_ingles'].map(mapa_niveis).fillna(0)
nivel_vaga_ingles_num = df_final_total['perfil_vaga_nivel_ingles'].map(mapa_niveis).fillna(0)

# Passo 3a: Criar a coluna 'match_ingles'
df_final_total['match_ingles'] = (nivel_candidato_ingles_num >= nivel_vaga_ingles_num).astype(int)

# --- Novas linhas para o Espanhol ---

# Passo 2b: Converter as colunas de Espanhol para números
nivel_candidato_espanhol_num = df_final_total['formacao_e_idiomas_nivel_espanhol'].map(mapa_niveis).fillna(0)
nivel_vaga_espanhol_num = df_final_total['perfil_vaga_nivel_espanhol'].map(mapa_niveis).fillna(0)

# Passo 3b: Criar a coluna 'match_espanhol'
df_final_total['match_espanhol'] = (nivel_candidato_espanhol_num >= nivel_vaga_espanhol_num).astype(int)

## match academico

# Passo 1: Criar o Mapa de Níveis Acadêmicos Ordenados
mapa_academico = {
    # Nível 0 - Nulos e Vazios
    np.nan: 0,
    '': 0,
    
    # Nível 1 a 3 - Fundamental
    'Ensino Fundamental Incompleto': 1,
    'Ensino Fundamental Cursando': 2,
    'Ensino Fundamental Completo': 3,

    # Nível 4 a 6 - Médio
    'Ensino Médio Incompleto': 4,
    'Ensino Médio Cursando': 5,
    'Ensino Médio Completo': 6,

    # Nível 7 a 9 - Técnico
    'Ensino Técnico Incompleto': 7,
    'Ensino Técnico Cursando': 8,
    'Ensino Técnico Completo': 9,

    # Nível 10 a 12 - Superior
    'Ensino Superior Incompleto': 10,
    'Ensino Superior Cursando': 11,
    'Ensino Superior Completo': 12,

    # Nível 13 a 15 - Pós-Graduação
    'Pós Graduação Incompleto': 13,
    'Pós Graduação Cursando': 14,
    'Pós Graduação Completo': 15,

    # Nível 16 a 18 - Mestrado
    'Mestrado Incompleto': 16,
    'Mestrado Cursando': 17,
    'Mestrado Completo': 18,

    # Nível 19 a 21 - Doutorado
    'Doutorado Incompleto': 19,
    'Doutorado Cursando': 20,
    'Doutorado Completo': 21
}


# Passo 2: Converter as colunas de texto para números usando o mapa
nivel_candidato_num = df_final_total['formacao_e_idiomas_nivel_academico'].map(mapa_academico).fillna(0)
nivel_vaga_num = df_final_total['perfil_vaga_nivel_academico'].map(mapa_academico).fillna(0)

# Passo 3: Comparar os níveis e criar a coluna 'match_academico'
df_final_total['match_academico'] = (nivel_candidato_num >= nivel_vaga_num).astype(int)

# Tratando a coluna 'informacoes_basicas_tipo_contratacao'

df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].fillna('').astype(str)
df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].str.replace(', ', ',') # Garante que não haja espaço após a vírgula
df_final_total['informacoes_basicas_tipo_contratacao'] = df_final_total['informacoes_basicas_tipo_contratacao'].str.strip() # Remove espaços no início/fim

# Realizando o label encoding na coluna no df_final_total 

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_tipo_contratacao'])[0]
df_final_total['informacoes_basicas_tipo_contratacao_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_prazo_contratacao'])[0]
df_final_total['informacoes_basicas_prazo_contratacao_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_prioridade_vaga'])[0]
df_final_total['informacoes_basicas_prioridade_vaga_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['informacoes_basicas_origem_vaga'])[0]
df_final_total['informacoes_basicas_origem_vaga_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_estado'])[0]
df_final_total['perfil_vaga_estado_codificada'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel profissional'])[0]
df_final_total['perfil_vaga_nivel profissional_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_academico'])[0]
df_final_total['perfil_vaga_nivel profissional_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_ingles'])[0]
df_final_total['perfil_vaga_nivel_ingles_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_nivel_espanhol'])[0]
df_final_total['perfil_vaga_nivel_espanhol_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['perfil_vaga_viagens_requeridas'])[0]
df_final_total['perfil_vaga_viagens_requeridas_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_academico'])[0]
df_final_total['formacao_e_idiomas_nivel_academico_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_ingles'])[0]
df_final_total['formacao_e_idiomas_nivel_ingles_codificado'] = nova_coluna_codificada

nova_coluna_codificada = pd.factorize(df_final_total['formacao_e_idiomas_nivel_espanhol'])[0]
df_final_total['formacao_e_idiomas_nivel_espanhol_codificado'] = nova_coluna_codificada

# Realizando o one-hot encoding na coluna 'perfil_vaga_areas_atuacao' e 'informacoes_profissionais_area_atuacao'

## 'perfil_vaga_areas_atuacao'
### PADRONIZAR OS SEPARADORES

def limpar_e_padronizar_separadores(texto):
    # Retorna uma string vazia se o texto for nulo/inválido
    if pd.isna(texto):
        return ""
    
    # Converte para string para garantir
    texto_str = str(texto)
    
    # Regra 1: Remove o '-' solto no final da string
    texto_limpo = re.sub(r'-$', '', texto_str.strip())
    
    # Regra 2: Substitui o '-' que está entre dois caracteres por ', '
    texto_limpo = re.sub(r'(\S)-(\S)', r'\1, \2', texto_limpo)
    
    return texto_limpo

### Aplicar a função e o one hot encoding

df_final_total['areas_atuacao_limpas'] = df_final_total['perfil_vaga_areas_atuacao'].apply(limpar_e_padronizar_separadores)
df_areas_codificadas = df_final_total['areas_atuacao_limpas'].str.get_dummies(sep=', ')
prefixo = 'perfil_vaga_areas_atuacao' + '_'
df_areas_codificadas = df_areas_codificadas.add_prefix(prefixo)
df_final_total = pd.concat([df_final_total, df_areas_codificadas], axis=1)

## 'informacoes_profissionais_area_atuacao'

df_final_temporario = df_final_total['informacoes_profissionais_area_atuacao'].str.get_dummies(sep=', ')
prefixo = 'informacoes_profissionais_area_atuacao' + '_'
df_final_temporario = df_final_temporario.add_prefix(prefixo)
df_final_total = pd.concat([df_final_total, df_final_temporario], axis=1)

# 1. Defina a função que calcula o match 
def calcular_match_profissional(row):
    str_vaga = row['areas_atuacao_limpas']
    str_candidato = row['informacoes_profissionais_area_atuacao']

    # Verifica se o valor é nulo e o transforma em string vazia se for o caso.
    if pd.isna(str_vaga):
        str_vaga = ''
    if pd.isna(str_candidato):
        str_candidato = ''

    set_vaga = set(item.strip() for item in str_vaga.split(',') if item.strip())
    set_candidato = set(item.strip() for item in str_candidato.split(',') if item.strip())

    if not set_vaga:
        return pd.Series([0, 1.0], index=['match_areas_contagem', 'match_areas_percentual'])

    competencias_em_comum = set_vaga.intersection(set_candidato)
    
    match_contagem = len(competencias_em_comum)
    match_percentual = match_contagem / len(set_vaga)
    
    return pd.Series([match_contagem, match_percentual], index=['match_areas_contagem', 'match_areas_percentual'])


# 2. Aplique a função a cada linha do DataFrame
match_features = df_final_total.apply(calcular_match_profissional, axis=1)

# 3. Junte as novas features ao seu DataFrame principal
df_final_total = pd.concat([df_final_total, match_features], axis=1)
df_final_total



--- DataFrame Final com as Novas Colunas de Match ---


,titulo,modalidade,id_vaga,n_prospect,dict_prospect_nome,dict_prospect_codigo,dict_prospect_situacao_candidado,dict_prospect_data_candidatura,dict_prospect_ultima_atualizacao,dict_prospect_comentario,dict_prospect_recrutador,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,informacoes_basicas_tipo_contratacao,informacoes_basicas_prazo_contratacao,informacoes_basicas_objetivo_vaga,informacoes_basicas_prioridade_vaga,informacoes_basicas_origem_vaga,informacoes_basicas_superior_imediato,informacoes_basicas_nome,informacoes_basicas_telefone,informacoes_basicas_data_inicial,informacoes_basicas_data_final,informacoes_basicas_nome_substituto,perfil_vaga_pais,perfil_vaga_estado,perfil_vaga_cidade,perfil_vaga_bairro,perfil_vaga_regiao,perfil_vaga_local_trabalho,perfil_vaga_vaga_especifica_para_pcd,perfil_vaga_faixa_etaria,perfil_vaga_horario_trabalho,perfil_vaga_nivel profissional,perfil_vaga_nivel_academico,perfil_vaga_nivel_ingles,perfil_vaga_nivel_espanhol,perfil_vaga_outro_idioma,perfil_vaga_areas_atuacao,perfil_vaga_principais_atividades,perfil_vaga_competencia_tecnicas_e_comportamentais,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,perfil_vaga_habilidades_comportamentais_necessarias,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,cv_pt,cv_en,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,infos_basicas_codigo_profissional,infos_basicas_nome,informacoes_pessoais_data_aceite,informacoes_pessoais_nome,informacoes_pessoais_cpf,informacoes_pessoais_fonte_indicacao,informacoes_pessoais_email,informacoes_pessoais_email_secundario,informacoes_pessoais_data_nascimento,informacoes_pessoais_telefone_celular,informacoes_pessoais_telefone_recado,informacoes_pessoais_sexo,informacoes_pessoais_estado_civil,informacoes_pessoais_pcd,informacoes_pessoais_endereco,informacoes_pessoais_skype,informacoes_pessoais_url_linkedin,informacoes_pessoais_facebook,informacoes_pessoais_download_cv,informacoes_profissionais_titulo_profissional,informacoes_profissionais_area_atuacao,informacoes_profissionais_conhecimentos_tecnicos,informacoes_profissionais_certificacoes,informacoes_profissionais_outras_certificacoes,informacoes_profissionais_remuneracao,informacoes_profissionais_nivel_profissional,informacoes_profissionais_qualificacoes,informacoes_profissionais_experiencias,formacao_e_idiomas_nivel_academico,formacao_e_idiomas_nivel_ingles,formacao_e_idiomas_nivel_espanhol,formacao_e_idiomas_outro_idioma,formacao_e_idiomas_instituicao_ensino_superior,formacao_e_idiomas_cursos,formacao_e_idiomas_ano_conclusao,formacao_e_idiomas_outro_curso,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato,match_academico,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,areas_atuacao_limpas,perfil_vaga_areas_atuacao_Administrativa,perfil_vaga_areas_atuacao_Comerci

In [7]:
# Tratando a coluna 'dict_prospect_status'

## Adicionando a coluna 'status_geral'
mapeamento_status_ajustado = {
    # ## Categoria Sucesso
    'Aprovado': 'Sucesso',
    'Contratado como Hunting': 'Sucesso',
    'Contratado pela Decision': 'Sucesso',
    'Documentação Cooperado': 'Sucesso',
    'Documentação CLT': 'Sucesso',
    'Documentação PJ': 'Sucesso',
    'Proposta Aceita': 'Sucesso',

    # ## Categoria Insucesso (agora inclui as desistências)
    'Não Aprovado pelo RH': 'Insucesso',
    'Não Aprovado pelo Cliente': 'Insucesso',
    'Não Aprovado pelo Requisitante': 'Insucesso',
    'Recusado': 'Insucesso',
    'Desistiu da Contratação': 'Insucesso', 
    'Desistiu': 'Insucesso',                
    'Sem interesse nesta vaga': 'Insucesso',

    # ## Categoria Andamento
    'Encaminhado ao Requisitante': 'Andamento',
    'Inscrito': 'Andamento',
    'Prospect': 'Andamento',
    'Entrevista Técnica': 'Andamento',
    'Em avaliação pelo RH': 'Andamento',
    'Entrevista com Cliente': 'Andamento',
    'Encaminhar Proposta': 'Andamento'
}

df_final_total['status_geral'] = df_final_total['dict_prospect_situacao_candidado'].map(mapeamento_status_ajustado)

## Removendo linhas com status 'Andamento' e NaN na coluna 'status_geral'
df_final_total = df_final_total.dropna(subset=['status_geral'])
condicao_para_manter = df_final_total['status_geral'] != 'Andamento'
df_final_total = df_final_total[condicao_para_manter]

## Realizando o label encoding na coluna 'status_geral'
nova_coluna_codificada = pd.factorize(df_final_total['status_geral'])[0]
df_final_total['status_geral_codificado'] = nova_coluna_codificada 

# Selecionando apenas as colunas para alimentar o modelo de ML

colunas_para_manter = [
    'id_vaga',
    'dict_prospect_codigo',
    'match_academico',
    'match_ingles',
    'match_espanhol',
    'match_areas_contagem',	
    'match_areas_percentual',
    'informacoes_basicas_tipo_contratacao_codificada',
    'informacoes_basicas_prazo_contratacao_codificada',
    'informacoes_basicas_prioridade_vaga_codificada',
    'informacoes_basicas_origem_vaga_codificada',
    'perfil_vaga_estado_codificada',
    'perfil_vaga_nivel profissional_codificado', # Atenção a este nome com espaço
    'perfil_vaga_nivel_ingles_codificado',
    'perfil_vaga_nivel_espanhol_codificado',
    'perfil_vaga_viagens_requeridas_codificado',
    'formacao_e_idiomas_nivel_academico_codificado',
    'formacao_e_idiomas_nivel_ingles_codificado',
    'formacao_e_idiomas_nivel_espanhol_codificado',
    'perfil_vaga_areas_atuacao_Administrativa',
    'perfil_vaga_areas_atuacao_Comercial',
    'perfil_vaga_areas_atuacao_Financeira/Controladoria',
    'perfil_vaga_areas_atuacao_Gestão e Alocação de Recursos de TI',
    'perfil_vaga_areas_atuacao_Jurídica',
    'perfil_vaga_areas_atuacao_Marketing',
    'perfil_vaga_areas_atuacao_Novos Negócios e Parcerias',
    'perfil_vaga_areas_atuacao_Qualidade Corporativa',
    'perfil_vaga_areas_atuacao_Recursos Humanos',
    'perfil_vaga_areas_atuacao_Relacionamento Técnico',
    'perfil_vaga_areas_atuacao_TI - Arquitetura',
    'perfil_vaga_areas_atuacao_TI - Banco de Dados',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Design',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Mobile',
    'perfil_vaga_areas_atuacao_TI - Desenvolvimento/Programação',
    'perfil_vaga_areas_atuacao_TI - Governança',
    'perfil_vaga_areas_atuacao_TI - Infraestrutura',
    'perfil_vaga_areas_atuacao_TI - Processos e Negócios',
    'perfil_vaga_areas_atuacao_TI - Projetos',
    'perfil_vaga_areas_atuacao_TI - Qualidade/Testes',
    'perfil_vaga_areas_atuacao_TI - SAP',
    'perfil_vaga_areas_atuacao_TI - Segurança da Informação',
    'perfil_vaga_areas_atuacao_TI - Sistemas e Ferramentas',
    'perfil_vaga_areas_atuacao_TI - Suporte',
    'perfil_vaga_areas_atuacao_TI - Telecom',
    'perfil_vaga_areas_atuacao_TI - UX',
    'informacoes_profissionais_area_atuacao_Administrativa',
    'informacoes_profissionais_area_atuacao_Comercial',
    'informacoes_profissionais_area_atuacao_Financeira/Controladoria',
    'informacoes_profissionais_area_atuacao_Gestão e Alocação de Recursos de TI',
    'informacoes_profissionais_area_atuacao_Jurídica',
    'informacoes_profissionais_area_atuacao_Marketing',
    'informacoes_profissionais_area_atuacao_Novos Negócios e Parcerias',
    'informacoes_profissionais_area_atuacao_Qualidade Corporativa',
    'informacoes_profissionais_area_atuacao_Recursos Humanos',
    'informacoes_profissionais_area_atuacao_Relacionamento Técnico',
    'informacoes_profissionais_area_atuacao_TI - Arquitetura',
    'informacoes_profissionais_area_atuacao_TI - Banco de Dados',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Design',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Mobile',
    'informacoes_profissionais_area_atuacao_TI - Desenvolvimento/Programação',
    'informacoes_profissionais_area_atuacao_TI - Governança',
    'informacoes_profissionais_area_atuacao_TI - Infraestrutura',
    'informacoes_profissionais_area_atuacao_TI - Processos e Negócios',
    'informacoes_profissionais_area_atuacao_TI - Projetos',
    'informacoes_profissionais_area_atuacao_TI - Qualidade/Testes',
    'informacoes_profissionais_area_atuacao_TI - SAP',
    'informacoes_profissionais_area_atuacao_TI - Segurança da Informação',
    'informacoes_profissionais_area_atuacao_TI - Sistemas e Ferramentas',
    'informacoes_profissionais_area_atuacao_TI - Suporte',
    'informacoes_profissionais_area_atuacao_TI - Telecom',
    'informacoes_profissionais_area_atuacao_TI - UX',
    'status_geral_codificado'
]

df_final_total = df_final_total[colunas_para_manter]
df_final_total

,id_vaga,dict_prospect_codigo,match_academico,match_ingles,match_espanhol,match_areas_contagem,match_areas_contagem,match_areas_percentual,match_areas_percentual,informacoes_basicas_tipo_contratacao_codificada,informacoes_basicas_prazo_contratacao_codificada,informacoes_basicas_prioridade_vaga_codificada,informacoes_basicas_origem_vaga_codificada,perfil_vaga_estado_codificada,perfil_vaga_nivel profissional_codificado,perfil_vaga_nivel_ingles_codificado,perfil_vaga_nivel_espanhol_codificado,perfil_vaga_viagens_requeridas_codificado,formacao_e_idiomas_nivel_academico_codificado,formacao_e_idiomas_nivel_ingles_codificado,formacao_e_idiomas_nivel_espanhol_codificado,perfil_vaga_areas_atuacao_Administrativa,perfil_vaga_areas_atuacao_Administrativa,perfil_vaga_areas_atuacao_Comercial,perfil_vaga_areas_atuacao_Comercial,perfil_vaga_areas_atuacao_Financeira/Controladoria,perfil_vaga_areas_atuacao_Financeira/Controladoria,perfil_vaga_areas_atuacao_Gestão e Alocação de Recursos de TI,perfil_vaga_areas_atuacao_Gestão e Alocação de Recursos de TI,perfil_vaga_areas_atuacao_Jurídica,perfil_vaga_areas_atuacao_Jurídica,perfil_vaga_areas_atuacao_Marketing,perfil_vaga_areas_atuacao_Marketing,perfil_vaga_areas_atuacao_Novos Negócios e Parcerias,perfil_vaga_areas_atuacao_Novos Negócios e Parcerias,perfil_vaga_areas_atuacao_Qualidade Corporativa,perfil_vaga_areas_atuacao_Qualidade Corporativa,perfil_vaga_areas_atuacao_Recursos Humanos,perfil_vaga_areas_atuacao_Recursos Humanos,perfil_vaga_areas_atuacao_Relacionamento Técnico,perfil_vaga_areas_atuacao_Relacionamento Técnico,perfil_vaga_areas_atuacao_TI - Arquitetura,perfil_vaga_areas_atuacao_TI - Arquitetura,perfil_vaga_areas_atuacao_TI - Banco de Dados,perfil_vaga_areas_atuacao_TI - Banco de Dados,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Design,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Design,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Mobile,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Mobile,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Programação,perfil_vaga_areas_atuacao_TI - Desenvolvimento/Programação,perfil_vaga_areas_atuacao_TI - Governança,perfil_vaga_areas_atuacao_TI - Governança,perfil_vaga_areas_atuacao_TI - Infraestrutura,perfil_vaga_areas_atuacao_TI - Infraestrutura,perfil_vaga_areas_atuacao_TI - Processos e Negócios,perfil_vaga_areas_atuacao_TI - Processos e Negócios,perfil_vaga_areas_atuacao_TI - Projetos,perfil_vaga_areas_atuacao_TI - Projetos,perfil_vaga_areas_atuacao_TI - Qualidade/Testes,perfil_vaga_areas_atuacao_TI - Qualidade/Testes,perfil_vaga_areas_atuacao_TI - SAP,perfil_vaga_areas_atuacao_TI - SAP,perfil_vaga_areas_atuacao_TI - Segurança da Informação,perfil_vaga_areas_atuacao_TI - Segurança da Informação,perfil_vaga_areas_atuacao_TI - Sistemas e Ferramentas,perfil_vaga_areas_atuacao_TI - Sistemas e Ferramentas,perfil_vaga_areas_atuacao_TI - Suporte,perfil_vaga_areas_atuacao_TI - Suporte,perfil_vaga_areas_atuacao_TI - Telecom,perfil_vaga_areas_atuacao_TI - Telecom,perfil_vaga_areas_atuacao_TI - UX,perfil_vaga_areas_atuacao_TI - UX,informacoes_profissionais_area_atuacao_Administrativa,informacoes_profissionais_area_atuacao_Administrativa,informacoes_profissionais_area_atuacao_Comercial,informacoes_profissionais_area_atuacao_Comercial,informacoes_profissionais_area_atuacao_Financeira/Controladoria,informacoes_profissionais_area_atuacao_Financeira/Controladoria,informacoes_profissionais_area_atuacao_Gestão e Alocação de Recursos de TI,informacoes_profissionais_area_atuacao_Gestão e Alocação de Recursos de TI,informacoes_profissionais_area_atuacao_Jurídica,informacoes_profissionais_area_atuacao_Jurídica,informacoes_profissionais_area_atuacao_Marketing,informacoes_profissionais_area_atuacao_Marketing,informacoes_profissionais_area_atuacao_Novos Negócios e Parcerias,informacoes_profissionais_area_atuacao_Novos Negócios e Parcerias,informacoes_profissionais_area_atuacao_Qualidade Corporativa,informacoes_profissionais_area_atuacao_Qualidade Corporativa,informaco

In [ ]:
# Separando am base de treino e teste

# A variável 'y' é a coluna que queremos prever (o nosso alvo).
y = df_final_total['status_geral_codificado']

colunas_para_remover = ['status_geral_codificado', 'id_vaga', 'dict_prospect_codigo']
X = df_final_total.drop(columns=colunas_para_remover)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Utilizar o smote para equilibrar as classes

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


In [ ]:
# rede neural (Mult Layer Perceptron)

# --- PASSO 1: Escalonamento dos Dados ---
# Criamos o objeto scaler
scaler = StandardScaler()

# Ajuste o scaler APENAS nos dados de treino
X_train = scaler.fit_transform(X_train)

# Use o scaler já ajustado para TRANSFORMAR os dados de teste
X_test = scaler.transform(X_test)

# --- PASSO 2: Construção da Arquitetura do Modelo ---
# O número de features de entrada é o número de colunas em X
n_features = X_train.shape[1]

# Usamos o modelo Sequential, que nos permite empilhar camadas uma a uma
modelo_nn = Sequential()

# Camada de Entrada e Primeira Camada Oculta
modelo_nn.add(Dense(64, activation='relu', input_shape=(n_features,)))
modelo_nn.add(Dropout(0.4)), # Desliga 50% dos neurônios para combater overfitting

# Segunda Camada Oculta
modelo_nn.add(Dense(32, activation='relu'))
modelo_nn.add(Dropout(0.2)), # Desliga 30% dos neurônios

modelo_nn.add(Dense(16, activation='relu')), # <-- Nova camada

# Camada de Saída
modelo_nn.add(Dense(1, activation='sigmoid'))

# --- PASSO 3: Compilação do Modelo ---
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
modelo_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostra um resumo da arquitetura criada
print("--- Arquitetura da Rede Neural ---")
modelo_nn.summary()



--- Arquitetura da Rede Neural ---


c:\Users\Usuário\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         7,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,561 (41.25 KB)

 Trainable params: 10,561 (41.25 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Configura o callback de Parada Antecipada
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# --- PASSO 4: Treinamento do Modelo ---
print("\n--- Iniciando o Treinamento da Rede Neural ---")
historico = modelo_nn.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_test, y_test),
    verbose=1 # verbose=1 mostra uma barra de progresso
)




--- Iniciando o Treinamento da Rede Neural ---
Epoch 1/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5895 - loss: 0.6780 - val_accuracy: 0.6545 - val_loss: 0.6327
Epoch 2/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6720 - loss: 0.6082 - val_accuracy: 0.6893 - val_loss: 0.6222
Epoch 3/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6874 - loss: 0.5893 - val_accuracy: 0.7053 - val_loss: 0.5947
Epoch 4/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6907 - loss: 0.5802 - val_accuracy: 0.7139 - val_loss: 0.5893
Epoch 5/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7021 - loss: 0.5662 - val_accuracy: 0.7053 - val_loss: 0.6017
Epoch 6/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6960 - loss: 0.5703 - val_accuracy: 0.7155 - val_loss: 0.5789
Epoch 7/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7092 - loss: 0.5533 - val_accuracy: 0.6926 - val_loss: 0.5921
Epoch 8/100
451/451 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/s

In [13]:
# --- PASSO 5: Avaliação Final do Modelo ---
print("\n--- Avaliação Final do Modelo nos Dados de Teste ---")

# .predict() retorna as probabilidades de pertencer à classe 1
probabilidades = modelo_nn.predict(X_test)
# Convertemos as probabilidades em classes (0 ou 1) usando um limiar de 0.5
y_pred_nn = (probabilidades > 0.5).astype(int)

print("\nRelatório de Classificação da Rede Neural:")
print(classification_report(y_test, y_pred_nn))


--- Avaliação Final do Modelo nos Dados de Teste ---
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Relatório de Classificação da Rede Neural:
              precision    recall  f1-score   support

           0       0.49      0.56      0.52       646
           1       0.83      0.79      0.81      1797

    accuracy                           0.73      2443
   macro avg       0.66      0.68      0.67      2443
weighted avg       0.74      0.73      0.74      2443



In [14]:
# 1. Obter as probabilidades usando o método .predict()

print("Gerando probabilidades com a Rede Neural...")
probabilidades_reprovado = modelo_nn.predict(X_test)

prob_ser_aprovado = 1 - probabilidades_reprovado.flatten()

# 3. Criar um novo DataFrame para o ranking
df_ranking = X_test.copy()

df_ranking = pd.DataFrame(X_test, columns=X.columns)

# 4. Adicionar as informações importantes para análise
df_ranking['probabilidade_match'] = prob_ser_aprovado
df_ranking['status_real'] = y_test # Adiciona o resultado real para comparação

# Para facilitar a identificação, vamos buscar os IDs originais usando o índice do DataFrame
df_ranking = df_ranking.join(df_final_total[['id_vaga', 'dict_prospect_codigo']])

# 5. Ordenar o ranking pela probabilidade, do maior para o menor
df_ranking_final = df_ranking.sort_values(by='probabilidade_match', ascending=False)
df_ranking_final = df_ranking_final[df_ranking_final['id_vaga'].notna()]

# 6. Exibir o resultado
print("\n--- Ranking de Candidatos (Gerado pela Rede Neural) ---")
colunas_para_exibir = ['id_vaga', 'dict_prospect_codigo', 'probabilidade_match', 'status_real']
df_ranking_final[colunas_para_exibir].head(20)

Gerando probabilidades com a Rede Neural...
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

--- Ranking de Candidatos (Gerado pela Rede Neural) ---


,id_vaga,dict_prospect_codigo,probabilidade_match,status_real
1326,5400.0,27230,0.999958,NaN
433,7927.0,32271,0.999875,NaN
63,4593.0,25365,0.995356,NaN
1992,6066.0,25395,0.991055,NaN
1765,5839.0,20225,0.991055,0.0
855,8349.0,26909,0.983273,NaN
434,7928.0,34316,0.979811,NaN
1028,8522.0,33996,0.958899,1.0
2053,6127.0,27425,0.952340,NaN
1884,5958.0,28270,0.950793,NaN
